<a href="https://colab.research.google.com/github/somas1/CT/blob/main/Pandas/Pandas_II_Student_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Pandas - Joins, Time Series & Rolling Statistics, SQL

## Tasks Today:

1) <b>Pandas</b> <br>
 &nbsp;&nbsp;&nbsp;&nbsp; a) Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - merge() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Breaking Down the Inner Join <br>
 &nbsp;&nbsp;&nbsp;&nbsp; b) Outer Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Use Case Example <br>
 &nbsp;&nbsp;&nbsp;&nbsp; c) Rolling Statistics and Time Series w/Pandas <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - read_csv() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - shift() <br>
 2) <b> SQL </b> <br>
      - Adding DataFrame to Database Table
      - Reading SQL Query Result into DataFrame

In [1]:
import pandas as pd
import numpy as np

### Joins <br>
<p>Very often you will have to combine data from several different sources to obtain the actual dataset you need for your exploration or modeling. Pandas draws heavily on SQL in its API design for joins, and if you are familiar with SQL joins, then Pandas will come naturally. Imagine you have two tables, one with customer name and gender, and the other with their purchases:</p>

In [2]:
# Generate some fake data
city_data = {
    1: 'Chicago',
    2: 'Boston',
    3: 'New York',
    4: 'San Francisco',
    5: 'Washington D.C.'
}

ages = np.random.randint(18,65,100)
city = [ city_data[num] for num in np.random.randint(1,6,100)]
customer_id = np.arange(1000,1100)


<p>Let's assume we have a dataset containing the customer ID's cities, and ages for a set of customers. Suppose further, that we have a second table, containing some customer ID's, and a feedback survey rating from those customers. Suppose finally, that you as an analyst are asked the following question: What city tends to give the best feedback for our product? We'll need a join operation to get the answer.</p>

In [3]:
customer_demographics = pd.DataFrame.from_dict({
    'ages': ages,
    'city': city,
    'customer_id': customer_id
})

customer_demographics

,ages,city,customer_id
0,58,Washington D.C.,1000
1,37,Washington D.C.,1001
2,49,Chicago,1002
3,51,San Francisco,1003
4,37,Chicago,1004
...,...,...,...
95,40,Boston,1095
96,42,Washington D.C.,1096
97,54,New York,1097
98,43,Washington D.C.,1098


In [4]:
# Generate some other fake data
customer_id = np.random.randint(1000,1200,100)
customer_rating = np.random.randint(1,11,100)

surveys = pd.DataFrame.from_dict({
    'customer_id': customer_id,
    'customer_rating': customer_rating
})

surveys.head(20)

,customer_id,customer_rating
0,1190,10
1,1107,3
2,1145,5
3,1063,9
4,1181,4
5,1172,9
6,1145,4
7,1103,9
8,1190,9
9,1145,2


##### merge()

<p>We can zip the two dataframes up with an inner join, using the Pandas API as follows:</p>

In [5]:
demo_ratings = customer_demographics.merge(surveys, on = 'customer_id', how = 'inner')
demo_ratings

,ages,city,customer_id,customer_rating
0,21,New York,1006,1
1,44,Chicago,1010,7
2,44,Chicago,1010,5
3,44,Chicago,1010,8
4,44,Chicago,1010,1
5,28,Boston,1015,10
6,48,New York,1017,2
7,48,New York,1017,1
8,21,Washington D.C.,1018,8
9,47,New York,1019,5


In [6]:
# drop duplicate customer_ids
demo_ratings_dropped = demo_ratings.drop_duplicates(subset = 'customer_id').reset_index(drop=True)
demo_ratings_dropped

,ages,city,customer_id,customer_rating
0,21,New York,1006,1
1,44,Chicago,1010,7
2,28,Boston,1015,10
3,48,New York,1017,2
4,21,Washington D.C.,1018,8
5,47,New York,1019,5
6,44,New York,1026,4
7,45,Boston,1027,3
8,38,San Francisco,1028,3
9,28,Chicago,1032,5


**.join() method**

In [8]:
# join() is used to combine two DataFrames on the index but not on
# columns whereas merge() is primarily used to specify the columns
# you wanted to join on, this also supports joining on indexes and combination of index and columns

demo_surveys_join = customer_demographics.join(other = surveys.set_index('customer_id'), on = 'customer_id', how = 'inner')
demo_surveys_join

demo_surveys_join_nodups = demo_surveys_join.drop_duplicates(subset = 'customer_id').reset_index(drop = True)
demo_surveys_join_nodups

,ages,city,customer_id,customer_rating
0,21,New York,1006,1
1,44,Chicago,1010,7
2,28,Boston,1015,10
3,48,New York,1017,2
4,21,Washington D.C.,1018,8
5,47,New York,1019,5
6,44,New York,1026,4
7,45,Boston,1027,3
8,38,San Francisco,1028,3
9,28,Chicago,1032,5


<p>Now it's a simple groupby operation to reveal the answer:</p>

But what if we didn't drop the customer_id duplicates...would we get the same city result??

What would be the total mean for the US customers?


##### Breaking Down the Inner Join <br>
<p>Let's take a closer look at the actual mechanics here however. There is some subtlety and nuance that is best understood by thinking of an inner join as two steps: a cartesian product, and then a filter. Observe the result of the following inner join:</p>

<p>Somehow, we ended up with more rows in the result than either of the original tables! This is because we joined on a $\textit{non-unique index}$. The join starts by computing all the possible combinations of rows, and then filters them based on the condition. Because of the duplicates, multiple combinations of rows make it through!</p>

### Outer Joins <br>
<p>An $\textit{outer join}$ produces all of the rows from the left table, and joins whatever rows it can to the right table, filling anywhere it cant with NaN's or None values. Below is a brief example, and then we will discuss a possible use case for this type of join.</p>

In [ ]:
# LEFT JOIN MERGE


In [ ]:
# RIGHT JOIN MERGE


##### Use Case Example

<p>As an example use case, let's imagine we have again the demographics and survey response tables from above:</p>

<p>Now, you are asked as an analyst to compare the demographics of survey responders to the demographics of all your customers combined.</p>

In [ ]:
# Count all survey responders against the total number of customers we have
# (Specifically the customers that haven't taken the survey yet)


# Producing similar result with .count()


In [ ]:
# average age of people that answered the survey


# demo_ratings_outer['customer_rating'] != 'N/A'


In [ ]:
# find the average age of people that did not answer the survey


### Rolling Statistics and Time Series with Pandas

##### read_csv()

##### shift() <br>
<p>Shift index by desired number of periods with an optional time freq</p>

In [ ]:
# Print all keys for easier use


In [ ]:
# create a new column that takes the difference in temperature from the previous day as an absolute value


In [ ]:
# Pip install for both psycopg2 and SQLAlchemy
import sys
!{sys.executable} -m pip install psycopg2-binary SQLAlchemy # the -binary is for mac users

In [ ]:
# databaselanguage(postgresql)://user:password@url:port/database_name



In [ ]:
# Writing a DataFrame to PostGreSQL database
# from sqlalchemy.types import Integer, Text,String,DateTime,Float
